In [11]:
import lore
from datamanager import *

from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, accuracy_score

from util import record2str

ipynb to py in pycharm:
- jupyter nbconvert --to script new_run.ipynb

## Dataset

* Here, I've used COMPAS Recidivism Risk Score data that have the both numerical and categorical features. 
* The task here to predict the class of a person among 3 classes (Low, Medium, High) 
* To convert categorical values to numerical, he basic strategy is to convert each category value into a new column and assigns a 1 or 0 (True/False) value to the column. This has the benefit of not weighting a value improperly but does have the downside of adding more columns to the data set.
* Pandas supports this feature using get_dummies. This function is named this way because it creates dummy/indicator variables (aka 1 or 0).


In [12]:
## Iris Dataset
# dataset_name = 'dataset/iris.csv'
# dataset = prepare_iris_dataset(dataset_name)

## wine
# dataset_name = 'dataset/wine.csv'
# dataset = prepare_wine_dataset(dataset_name)

##############################################
#           Categorical dataset              #
##############################################
## german: (0 = Good, 1 = Bad)
# dataset_name = 'dataset/german_credit.csv'
# dataset = prepare_german_dataset(dataset_name)

## adult: ['<=50K', '>50K']
# dataset_name = 'dataset/adult.csv'
# dataset = prepare_adult_dataset(dataset_name)

## compas-scores-two-years: ['High', 'Low', 'Medium']
dataset_name = 'dataset/compas-scores-two-years.csv'
dataset = prepare_compass_dataset(dataset_name)

dataframe = dataset[0]
class_name = dataset[1]
dataset_fin = prepare_dataset(dataframe, class_name)

In [13]:
df = dataset_fin[0] #dataframe with unique numeric class values(0, 1, ...)
feature_names = dataset_fin[1]
class_values = dataset_fin[2]
numeric_columns = dataset_fin[3]
rdf = dataset_fin[4] #real dataframe
real_feature_names = dataset_fin[5]
features_map = dataset_fin[6] #map each class name to its unique numeric value

In [14]:
rdf.head()

,age,priors_count,days_b_screening_arrest,is_recid,is_violent_recid,two_year_recid,length_of_stay,age_cat,sex,race,c_charge_degree,class
0,69,0,1,0,0,0,0,Greater than 45,Male,Other,F,Low
1,34,0,1,1,1,1,10,25 - 45,Male,African-American,F,Low
2,24,4,1,1,0,1,1,Less than 25,Male,African-American,F,Low
3,23,1,1,0,0,0,0,Less than 25,Male,African-American,F,High
4,43,2,1,0,0,0,0,25 - 45,Male,Other,F,Low


In [15]:
df.head()

,age,priors_count,days_b_screening_arrest,is_recid,is_violent_recid,two_year_recid,length_of_stay,age_cat=25 - 45,age_cat=Greater than 45,age_cat=Less than 25,...,sex=Male,race=African-American,race=Asian,race=Caucasian,race=Hispanic,race=Native American,race=Other,c_charge_degree=F,c_charge_degree=M,class
0,69,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,1,0,1
1,34,0,1,1,1,1,10,1,0,0,...,1,1,0,0,0,0,0,1,0,1
2,24,4,1,1,0,1,1,0,0,1,...,1,1,0,0,0,0,0,1,0,1
3,23,1,1,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,1,0,0
4,43,2,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,1,0,1


## Black box classifier

* Split dataset to train and test sets, then train a Random Forest classifier

In [16]:
X = df.loc[:, df.columns != class_name].values
y = df[class_name].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
blackbox = RandomForestClassifier()
blackbox.fit(X_train, y_train)

RandomForestClassifier()

In [17]:
y_pred = blackbox.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Accuracy: 0.611


## select an instance _x_

In [18]:
i = 10
x = X_test[i]
y_val = blackbox.predict(x.reshape(1,-1))[0]

print(class_values)
class_prob = blackbox.predict_proba(x.reshape(1,-1))[0]
print(class_prob)

y_val_name = class_values[y_val]
print('blackbox(x) = { %s }' % y_val_name)


['High', 'Low', 'Medium']
[0.51 0.12 0.37]
blackbox(x) = { High }


In [19]:
print('x = %s' % record2str(x, feature_names, numeric_columns))

x = { age = 30, priors_count = 7, days_b_screening_arrest = 1, is_recid = 1, is_violent_recid = 0, two_year_recid = 1, length_of_stay = 38, age_cat = 25 - 45, sex = Male, race = African-American, c_charge_degree = F }


# LORE explainer (explaining an instance x)

In [20]:
lore_obj = lore.LORE(X_test, blackbox, feature_names, class_name, class_values,
                 numeric_columns, features_map, neigh_type='ngmusx', verbose=False)

In [21]:
# just to check
Z = lore_obj.neighgen_fn(x)
print('Z is:',Z)
Z.shape

Z is: [[ 3.00000000e+01  7.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 3.00525416e+01  6.96264528e+00  1.03888026e+00 ... -6.34152520e-02
   1.08691735e+00  3.26671362e-02]
 [ 3.00250047e+01  6.93132453e+00  1.01203076e+00 ...  8.04886974e-02
   9.92041676e-01 -1.96186784e-02]
 ...
 [ 2.97290065e+01  6.96376741e+00  9.19541222e-01 ...  6.27296964e-02
   1.00571590e+00 -3.02542463e-01]
 [ 2.97352323e+01  6.87140497e+00  1.01651406e+00 ...  8.27060910e-02
   9.87475054e-01 -2.69209195e-01]
 [ 2.97848204e+01  6.93910569e+00  1.07670989e+00 ...  5.78003749e-02
   8.65814638e-01 -2.65006573e-01]]


(1010, 20)

* We can see here the factual and counter-factual rules obtained by our LORE explainer

In [27]:
explanation = lore_obj.explain_instance(x, samples=1000, nbr_runs=10)

print(explanation)

r = { length_of_stay > 37.93, race = African-American } --> { class: High }
c = { { length_of_stay <= 37.93, sex != Male },
      { length_of_stay <= 37.93, priors_count > 7.50 },
      { length_of_stay <= 37.93, two_year_recid <= 0.50 },
      { length_of_stay <= 37.93, race = Other } }


* The selected instance itself:

In [30]:
print('x = %s' % record2str(x, feature_names, numeric_columns))

x = { age = 30, priors_count = 7, days_b_screening_arrest = 1, is_recid = 1, is_violent_recid = 0, two_year_recid = 1, length_of_stay = 38, age_cat = 25 - 45, sex = Male, race = African-American, c_charge_degree = F }


## check the borderline

* now I just test counter-factual rules by changing some of the instance x feature values

In [28]:
temp_x = x.copy()

In [29]:
features_map

defaultdict(dict,
            {0: {'age': 0},
             1: {'priors_count': 1},
             2: {'days_b_screening_arrest': 2},
             3: {'is_recid': 3},
             4: {'is_violent_recid': 4},
             5: {'two_year_recid': 5},
             6: {'length_of_stay': 6},
             7: {'25 - 45': 7, 'Greater than 45': 8, 'Less than 25': 9},
             8: {'Female': 10, 'Male': 11},
             9: {'African-American': 12,
              'Asian': 13,
              'Caucasian': 14,
              'Hispanic': 15,
              'Native American': 16,
              'Other': 17},
             10: {'F': 18, 'M': 19}})

In [40]:
# c = { length_of_stay <= 37.93, sex != Male }
temp_x[6] = 37
temp_x[10] = 1
temp_x[11] = 0
print('x = %s' % record2str(temp_x, feature_names, numeric_columns))

x = { age = 30, priors_count = 7, days_b_screening_arrest = 1, is_recid = 1, is_violent_recid = 0, two_year_recid = 1, length_of_stay = 37, age_cat = 25 - 45, sex = Female, race = African-American, c_charge_degree = F }


In [41]:
print(class_values)
print(blackbox.predict_proba(temp_x.reshape(1,-1))[0])
print(class_values[blackbox.predict(temp_x.reshape(1,-1))[0]])

['High', 'Low', 'Medium']
[0.32 0.13 0.55]
Medium


In [43]:
explanation_temp = lore_obj.explain_instance(temp_x, samples=1000, nbr_runs=10)

print(explanation_temp)

r = { age_cat = 25 - 45, age_cat != Less than 25, sex = Female } --> { class: Medium }
c = { { age_cat != 25 - 45 },
      { age_cat = Less than 25 } }


* if I would change the age_cat value, I have:

In [45]:
#c = { age_cat = Less than 25 }
temp_x[9] = 1
temp_x[7] = 0
print('x = %s' % record2str(temp_x, feature_names, numeric_columns))

x = { age = 30, priors_count = 7, days_b_screening_arrest = 1, is_recid = 1, is_violent_recid = 0, two_year_recid = 1, length_of_stay = 37, age_cat = Less than 25, sex = Female, race = African-American, c_charge_degree = F }


In [46]:
print(class_values)
print(blackbox.predict_proba(temp_x.reshape(1,-1))[0])
print(class_values[blackbox.predict(temp_x.reshape(1,-1))[0]])

['High', 'Low', 'Medium']
[0.66 0.04 0.3 ]
High


In [50]:
explanation_temp2 = lore_obj.explain_instance(temp_x, samples=1000, nbr_runs=10)

print(explanation_temp2)

r = { age > 29.83 } --> { class: High }
c = { { age <= 29.83, age_cat = 25 - 45 } }


In [51]:
explanation_temp2 = lore_obj.explain_instance(temp_x, samples=1000, nbr_runs=10)

print(explanation_temp2)

r = { age_cat != 25 - 45, is_recid > 0.91, race = African-American } --> { class: High }
c = { { age_cat = 25 - 45, age_cat != Less than 25 },
      { race != African-American, age_cat != Less than 25 },
      { age_cat = 25 - 45, race = Other } }
